# Seleção evolutiva de conjunto de classificadores multirrótulos
    Felipe Carneiro Machado - 14569373
    Thales Sena de Queiroz - 
    Bruno Lima -

Realizado como projeto final da disciplina SCC0911 - Computação Bionispirada

Baseado no artigo: "An evolutionary approach to build ensembles of multi-label classifiers"
Por: Jose M. Moyano, Eva L. Gibaja, Krzysztof J. Cios, Sebastián Ventura

## Introdução
O problema da classificação multirrótulos é mais complexo que o que aparenta de imediato, visto que não deve considerar apenas as relações entre *features* mas também a relação entre rótulos.

Uma das técnicas clássicas para realizar tal tarefa que identifica relações entre rótulos é o *Label Powerset* (LP), que transforma um problema multirrótulos em um problema multiclasses, onde cada classe corresponde a um elemento do *Powerset* (conjunto dos subconjuntos) do conjunto dos rótulos.

Entretanto, a cardinalidade do *Powerset* de um conjunto $\mathcal{L}=\{\lambda_1, \lambda_2, ..., \lambda_q,\}$ é dada por
$|\mathcal{P}(\mathcal{L})|=2^{q-1}$
Ou seja, a complexidade assintótica da quantidade de rótulos é da ordem exponencial, desse modo, o método se torna inviável para problemas com quantidades muito grandes de rótulos. Os datasets abordados neste projeto possuem 12 e 8 rótulos, o que já é suficiente para procurar alternativas. 

## Proposta
Os autores do artigo no qual nos baseamos proprõem um uso alternativo do LP, em sua abordagem, um classificador é composto de $q$ classificadores multiclasse, cada um responsável por exatamente $k$ rótulos do conjunto, convertidos em problemas multiclasse pelo uso do LP.

Segue a ilustração de um conjunto desse tipo, com $q=4$ e $k=2$: 

| |$\lambda_1$   | $\lambda_2$   | $\lambda_3$   | $\lambda_4$   |
|------------|------------|------------|------------|------------|
| C1  | 0  | 1  | 1  | 0  |
| C2  | 1  | 1  | 0  | 0  |
| C3  | 0  | 0  | 1  | 1  |
| C4  | 0  | 1  | 0  | 1  |

Onde 1 representa que o classificador está responsável por um rótulo, e 0 o contrário.

Para obter um resultado final, o conjunto decide pelo "voto" da maioria, isto é, para cada rótulo $\lambda_q$, sua presença ou ausência é determinada pelo resultado da maioria dos classificadores associados a este rótulo.

Além disso, os autores propõem o uso de algoritmos evolutivos genéticos para encontrar o melhor conjunto de classificadores para um determinado problema. 

## Algoritmo Evolutivo

### Indivíduo
O indivíduo é o próprio classificador, representado genomicamente por uma matrix (ou vetor) binário, de forma análoga à mostrada na seção acima. No artigo, utiliza-se como classificador multiclasses árvores de decisão construídas com o algoritmo C4.5, porém utilizaremos o algoritmo CART, para manter padrões, por ser o implementado pela biblioteca *scikit-learn*.

### Função *Fitness*
A função *Fitness* considera tanto a precisão da classificação quanto a cobertura do conjunto de rótulos, através de medidas definidas a seguir.

A primeira é a FMeasure da previsão definida por
\begin{equation}
FMeasure=\frac{1}{m}\sum_{i=0}^{m}2\frac{\langle Y_i, \hat{Y_i}\rangle}{\langle Y_i, Y_i \rangle + \langle \hat{Y_i}, \hat{Y_i}\rangle} 
\end{equation}

Onde $m$ é o número de observações do conjunto teste, $Y_i$ representa a i-ésima observação do conjunto de teste, e $\hat{Y_i}$ representa a i-ésima predição do conjunto de classificadores. A FMeasure está definida no intervalo [0, 1] e busca-se maximizá-la. 

A segunda é o coeficiente de cobertura, que visa quantificar a homogeinidade na distribuição de rótulos por cada conjunto de classificadores. Pela maneira como o indivíduo foi definido, sempre haverá o mesmo número de rótulos avaliados em um conjunto,
mas sua distribuição varia.

O coeficiente de cobertura é definido como:
\begin{equation}
C_r=\frac{\sigma_{individual}}{\sigma_{worst case}}
\end{equation}

Onde $\sigma_{individual}$ é p desvio padrão do vetor de cobertura do indivíduo e $\sigma_{worst case}$ é o desvio padrão no pior caso.

O vetor de cobertura contém a contagem de aparições de um rótulo no conjunto. Tome como exemplo o indivíduo mostrado na seção de Introdução, seu vetor de cobertura é [1, 3, 2, 2] e o pior caso nas suas condições seria [4, 4, 0, 0].

O coeficiente de cobertura também está restrito ao intervalo [0, 1] e deve ser minimizado.

Por fim, a função *fitness* proposta pelo artigo é dada por:
\begin{equation}
fitness=\frac{Fmeasure + (1 - C_r)}{2}
\end{equation}

Estando restrita ao intervalo [0, 1] e devendo ser maximizada.

### *Cross-over*
Foi utilizado o operador de *cross-over* uniforma, onde são trocadas as linhas das matrizes (genoma) entre dois indivíduos seguindo uma dada probabilidade. Como trocar linhas do genoma corresponde a trocar um classificador, todos os indivíduos gerados são válidos

### Mutação baseada em correlação
A mutação corresponde em, dentro de um classificador do indivíduo, trocar de lugar um 1 e um 0. O 1 é escolhido aleatoriamente, porém o 0 terá probabilidade de ser escolhido baseado no coeficiente $\phi$ de correlação entre rótulos. O qual está limitado a [0, 1] com -1 singificando correlação inversa total, 0 correlação nula e 1, correlação direta total.

Assim, o peso de escolha de cada 0, com posição representada por b, do classificador é dado por:
\begin{equation}
w_b = \epsilon + \sum_{l \in A}|\phi_{b, l}|
\end{equation}

Onde $A$ é conjunto de rótulos ativos no classificador e $\phi_{b, l}$ é o coeficiente $\phi$ de correlação entre os rótulos $b$ e $l$ e $\epsilon$ é um pequeno valor que permite que rótulos descorrelacionados também possam ser escolhidos. O módulo é utilizado no coeficiente $\phi$ pois busca-se apenas verficicar sua correlação, independente de sua natureza.

### Fluxo do algoritmo evolutivo
O algoritmo evolutivo usado é elitista, descartando o melhor indivíduo de uma geração apenas se houver um melhor na próxima.

Também utiliza a seleção de torneio para selecionar a próxima geração, sobre a qual serão aplicados os operadores de *cross-over* e mutação.

Segue abaixo o fluxo do algoritmo em pseudo-código:
```python
population = generate_random_population(pop_size)
for g in max_iterations:
    evaluate(population)
    best_individual = best(population)
    new_population = select_tournament(population, pop_size)
    for indvidual in new_population:
        cross_over(individual, choice(population))
        mutate(individual)
    evaluate(new_population)
    if best_individual.fitness > best(new_population).fitness:
        worst(new_population) = best_individual
    population = new_population
```
    
Segue abaixo a implementação do algoritmo para o dataset de plantas.

In [43]:
# Dependencias
%pip install numpy pandas scikit-learn scikit-multilearn

Note: you may need to restart the kernel to use updated packages.


In [44]:
# Bibliotecas utilizadas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import LabelPowerset
from sklearn.tree import DecisionTreeClassifier
from random import shuffle, uniform, randint, choice, choices
from copy import deepcopy
from sklearn.metrics import hamming_loss

# Para implementação foi escolhido o sklearn como framework principal e o skmultilearn para utilizar o Label Powerset

# Leitura e tratamento de dados a partir do csv

df = pd.read_csv("Plants_Dataset_Term_Frequency.tsv", delimiter="\t")

# Rotulos
target = df[['ChloroplastProteins', 'CytoplasmProteins',       'EndoplasmicReticulumProteins', 'ExtracellProteins','GolgiApparatusProteins', 'MitochondrionProteins', 'NucleusProteins','PeroxisomeProteins', 'PlastidProteins', 'VacuoleProteins', "CellMembraneProteins", "CellWallProteins"]]    

# Features
inp = df.drop(columns=['ChloroplastProteins', 'CytoplasmProteins',
       'EndoplasmicReticulumProteins', 'ExtracellProteins',
       'GolgiApparatusProteins', 'MitochondrionProteins', 'NucleusProteins',
       'PeroxisomeProteins', 'PlastidProteins', 'VacuoleProteins', "CellMembraneProteins", "CellWallProteins", "GO_ID"])

inp = inp.to_numpy()
target = target.to_numpy()

# Separacao em teste e tein
xtrain, xtest, ytrain, ytest = train_test_split(inp, target, test_size=0.3, train_size=0.7)

In [ ]:
# Constantes do codigo

n = 12 # Numero total de labels
k = 6 # Labels por classificador
q = 8 # Classificadores por individuo
pop_size = 50 # Tamanho da população
stdvw = np.std(np.array([q] * k + [0] * (n-k))) # Desvio padrao no pior caso do vetor de cobertura
cross_over_rate = 0.5 # Taxa de cross-over (dentro do individuo)
cross_over_chance = 0.9 # Probabilidade de haver cross-over
mutation_rate = 0.2 # Taxa de mutação
max_gen = 100 # Maximo de iteracoes

# Tabela hash para evitar que conjuntos que ja foram avaliados o sejam novamente
table = {}

In [46]:
# Inicio das funcoes do algoritmo evolutivo

# Computa os valores de correlacao phi entre cada par de rotulos
def compute_phi_matrix():
    matrix = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        ni_1_ = np.sum(ytrain[:, i])
        for j in range(i + 1, n):
            nj_1_ = sum(ytrain[:, j])
            nij_11 = sum((ytrain[:, j]+ytrain[:, i]) == 2)
            nt = ytrain.shape[0]
            matrix[i, j] = (nt * nij_11 - nj_1_ * ni_1_) / np.sqrt(ni_1_ * nj_1_ * (nt - ni_1_) * (nt - nj_1_))
            matrix[j, i] = matrix[i, j]

    return matrix
# Acessada como variavel global      
phi = compute_phi_matrix()

In [47]:
# Funcoes para manipulacao dos conjuntos/individuos

# Funcao hash para identificar cada conjunto
def hash_ensemble(ensemble):
    h = ()
    for clf in ensemble:
        clf[1].sort()
        h += tuple(clf[1])
    return h

# Transforma uma matriz (genoma) em um conjunto de classificadores
def matrix_to_ensemble(matrix):
    ensemble : list[LabelPowerset]= []
    for row in matrix:
        l = []
        for i in range(len(row)):
            if row[i] == 1:
                l.append(i)
        ensemble.append((
            LabelPowerset(classifier=DecisionTreeClassifier(), require_dense=[True, True])
            .fit(xtrain, ytrain[:, l]),
            l
        ))
    return ensemble

# Gera uma matriz que seja um genoma valido
def random_matrix():
    m = [None] * q
    for i in range(q):
        l = [1] * k + [0] * (n - k)
        shuffle(l)
        m[i] = l
    return m

# Transforma um conjunto de classificadores para uma matriz/genoma
def ensemble_to_matrix(ensemble : list[tuple[LabelPowerset, list[int]]]):
    matrix = np.zeros((q, n))
    for i, clf in enumerate(ensemble):
        matrix[i, clf[1]] = 1
    return matrix

In [48]:
# Funcao de predicao do conjunto
def ensemble_predict(ensemble : list[tuple[LabelPowerset, list[int]]]):
    result = []
    for row in xtest: # Itera pelas observacoes
        matrix = np.ones((q, n)) * -1 # Armazena as predicoes de cada classificador
        for i, clf in enumerate(ensemble): # Para cada classificador
            matrix[i, clf[1]] = clf[0].predict(row.reshape(1, -1)).toarray() 
        arr = np.zeros((n,))
        i = 0
        # Realiza a "votacao" dos classificadores
        for col in matrix.T:
            total = 0
            ones = 0
            for p in col:
                if p != -1:
                    total += 1
                    ones += p
            if total == 0:
                arr[i] = randint(0, 1)
            else:
                arr[i] = 1 if ones / total > 0.5 else 0
            i += 1
        result.append(arr)   
    # Retorna um unico vetor de rotulos
    return np.array(result)

In [49]:
# Funcoes para calculo do fitness


# Coeficiente de cobertura
def coverage_ratio(ensemble : list[tuple[LabelPowerset, list[int]]]):
    arr = np.zeros((n,))
    for clf in ensemble:
        for i in clf[1]:
            arr[i] += 1
    return np.std(arr)/stdvw

# Calculo da FMeasure
def prediction_fscore(prediction : np.ndarray):
    exf = 0
    for i, row in enumerate(prediction):
        exf += 2 *(row @ ytest[i]) / ((row @ row) + (ytest[i] @ ytest[i]))
    exf = exf/ ytest.shape[0]
    return exf

# Calculo da fitness
def fitness(ensemble, alpha = 0.7):
    if table.get(hash_ensemble(ensemble), False): # Acessa a hash table em busca do conjunto
        return table.get(hash_ensemble(ensemble), False)
    prediction = ensemble_predict(ensemble)
    fscore = prediction_fscore(prediction)
    cr = coverage_ratio(ensemble)
    fit = (alpha * fscore + (1 - cr) * (1-alpha)) / 2
    table[hash_ensemble(ensemble)] = fit # Armazena o fitness na hash table
    return fit

In [50]:
# Operadores geneticos

# Cross-over uniforme, retorna mas tambem realiza mudancas in-place
def uniform_crossover(ensemble1, ensemble2):
    for i in range(q):
        if uniform(0, 1) < cross_over_rate:
            temp = ensemble1[i]
            ensemble1[i] =  ensemble2[i]
            ensemble2[i] = temp
    return ensemble1, ensemble2

# Epsilon da equacao de pesos para mutacao
epsilon = 0.1
# Funcao de mutacao
def mutate(ensemble : list[tuple[LabelPowerset, list[int]]]):
    clf = randint(0, q - 1) # Escolhe o classificador a ser mutado
    i1 = choice(ensemble[clf][1]) # Escolhe um rotulo ativo
    weights = [0] * n
    for i in range(n): # Constroi o vetor de pesos
        if not i in ensemble[clf][1]:
            acum = 0
            for j in ensemble[clf][1]:
                acum += abs(phi[i][j])                
            weights[i] = epsilon + acum
    swap = choices(list(range(n)), weights, k=1)[0] # Escolhe label inativa
    # Realiza a troca
    ensemble[clf][1].remove(i1)
    ensemble[clf][1].append(swap)
    # Refaz o individuo
    return matrix_to_ensemble(ensemble_to_matrix(ensemble))

# Selecao de torneio
def tournament(pop, fits,k=2, p=1):
    chosen = choices(list(range(pop_size)), k=k)
    return pop[chosen[0]] if fits[chosen[0]] > fits[chosen[1]] else pop[chosen[1]]

In [ ]:
# Execucao do algoritmo evolutivo

pop = [random_matrix() for _ in range(pop_size)] # Geracao aleatoria de genomas
pop_ensembles = list(map(matrix_to_ensemble, pop)) # Gera individuos a partir do genoma
fits = [fitness(e) for e in pop_ensembles] # Calculo de fitness
for g in range(max_gen):
    # Armazena o melhor individuo
    besti = np.argmax(fits)
    beste = deepcopy(pop_ensembles[besti])
    bestf = fits[besti]
    # Nova populacao selecionada a partir de torneio
    s = [tournament(pop_ensembles, fits) for _ in range(pop_size)]
    count = 0
    newpop = [None] * pop_size
    print(f"Generation {g}: best fitness = {bestf:.4f} Mean fitness = {np.mean(fits)}")
    # Reconstroi a populacao atraves de cross-over e mutacao
    while count < pop_size:
        for i, e in enumerate(s):
            p = uniform(0, 1)
            if p < cross_over_rate and count < pop_size - 1 and i < pop_size - 1:
                newpop[count], newpop[count + 1] = uniform_crossover(e, s[i + 1])
                count += 2
            if p < mutation_rate and count < pop_size:
                newpop[count] = mutate(e)
                count += 1
            if count == pop_size:
                break
    # Reavaliacao da populacao
    newfits = [fitness(e) for e in newpop]
    newbesti = np.argmax(newfits) # Novo Melhor
    worsti = np.argmin(newfits) # Novo pior
    if bestf > newfits[newbesti]: # se novo melhor melhor que antigo melhor troca o antigo com o pior
        newfits[worsti] = bestf
        newpop[worsti] = beste
    fits = newfits 
    pop_ensembles = newpop
    
bi = np.argmax(fits)
best_ensemble = pop_ensembles[bi]
print("hamming loss")
print(hamming_loss(ytest, ensemble_predict(best_ensemble)))
print("matrix")
print(ensemble_to_matrix(best_ensemble)) 

A execução do algoritmo evolutivo é bastante demorada, assim, segue abaixo uma matriz resultado da sua execução, que pode ser facilmente transformada em classificador novamente

In [41]:
matrix = eval("""list([[0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1.],
 [1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.],
 [0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0.],
 [0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0.],
 [0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.],
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.],
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1.],
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1.]])
""".replace(".", ","))
ensemble = matrix_to_ensemble(matrix)
print(hamming_loss(ytest, ensemble_predict(ensemble)))

0.011781609195402299


## Dataset de vírus
Utilizaremos a mesma abordagem para o segundo dataset do trabalho

In [54]:
# Obtencao dos dados

df = pd.read_csv("Virus_Dataset_Term_Frequency.tsv", delimiter="\t")

df

# # Rotulos
target = df[["ViralCapsidProteins","HostCellMembraneProteins", "HostEndoplasmicReticulumProteins", "HostCytoplasmProteins", "HostNucleusProteins", "SecretedProteins"]]    

# # Features
inp = df.drop(columns=["ViralCapsidProteins","HostCellMembraneProteins", "HostEndoplasmicReticulumProteins", "HostCytoplasmProteins", "HostNucleusProteins", "SecretedProteins", "GO_ID"])

inp = inp.to_numpy()
target = target.to_numpy()

# # Separacao em teste e tein
xtrain, xtest, ytrain, ytest = train_test_split(inp, target, test_size=0.3, train_size=0.7)



# Redefinicao de constantes

n = 6 # Numero total de labels
k = 3 # Labels por classificador
q = 6 # Classificadores por individuo
pop_size = 50 # Tamanho da população
stdvw = np.std(np.array([q] * k + [0] * (n-k))) # Desvio padrao no pior caso do vetor de cobertura
cross_over_rate = 0.5 # Taxa de cross-over (dentro do individuo)
cross_over_chance = 0.9 # Probabilidade de haver cross-over
mutation_rate = 0.2 # Taxa de mutação
max_gen = 100 # Maximo de iteracoes

# Tabela hash para evitar que conjuntos que ja foram avaliados o sejam novamente
table = {}

# Matriz de correlacoes
phi = compute_phi_matrix()


In [55]:
# Execucao do algoritmo evolutivo

pop = [random_matrix() for _ in range(pop_size)] # Geracao aleatoria de genomas
pop_ensembles = list(map(matrix_to_ensemble, pop)) # Gera individuos a partir do genoma
fits = [fitness(e) for e in pop_ensembles] # Calculo de fitness
for g in range(max_gen):
    # Armazena o melhor individuo
    besti = np.argmax(fits)
    beste = deepcopy(pop_ensembles[besti])
    bestf = fits[besti]
    # Nova populacao selecionada a partir de torneio
    s = [tournament(pop_ensembles, fits) for _ in range(pop_size)]
    count = 0
    newpop = [None] * pop_size
    print(f"Generation {g}: best fitness = {bestf:.4f} Mean fitness = {np.mean(fits)}")
    # Reconstroi a populacao atraves de cross-over e mutacao
    while count < pop_size:
        for i, e in enumerate(s):
            p = uniform(0, 1)
            if p < cross_over_rate and count < pop_size - 1 and i < pop_size - 1:
                newpop[count], newpop[count + 1] = uniform_crossover(e, s[i + 1])
                count += 2
            if p < mutation_rate and count < pop_size:
                newpop[count] = mutate(e)
                count += 1
            if count == pop_size:
                break
    # Reavaliacao da populacao
    newfits = [fitness(e) for e in newpop]
    newbesti = np.argmax(newfits) # Novo Melhor
    worsti = np.argmin(newfits) # Novo pior
    if bestf > newfits[newbesti]: # se novo melhor melhor que antigo melhor troca o antigo com o pior
        newfits[worsti] = bestf
        newpop[worsti] = beste
    fits = newfits 
    pop_ensembles = newpop
    
bi = np.argmax(fits)
best_ensemble = pop_ensembles[bi]
print("hamming loss")
print(hamming_loss(ytest, ensemble_predict(best_ensemble)))
print("matrix")
print(ensemble_to_matrix(best_ensemble))    

Generation 0: best fitness = 0.4625 Mean fitness = 0.42624585093010936
Generation 1: best fitness = 0.4625 Mean fitness = 0.42448845769327515
Generation 2: best fitness = 0.4625 Mean fitness = 0.40596009886064005
Generation 3: best fitness = 0.4801 Mean fitness = 0.41670509437828607
Generation 4: best fitness = 0.4801 Mean fitness = 0.4070943899967216
Generation 5: best fitness = 0.4801 Mean fitness = 0.41807371923733294
Generation 6: best fitness = 0.4801 Mean fitness = 0.4293202373539922
Generation 7: best fitness = 0.4801 Mean fitness = 0.3713881983113154
Generation 8: best fitness = 0.4801 Mean fitness = 0.4124807647544573
Generation 9: best fitness = 0.4801 Mean fitness = 0.3815867753169531
Generation 10: best fitness = 0.4801 Mean fitness = 0.38271607076750724
Generation 11: best fitness = 0.4801 Mean fitness = 0.39109554624979664
Generation 12: best fitness = 0.4801 Mean fitness = 0.4185655381217679
Generation 13: best fitness = 0.4801 Mean fitness = 0.41558020137974416
Generati

Novamente, a execução de algoritmo é bastante lenta, assim, sua matriz resultado foi utilizada abaixo.

In [95]:
matrix = eval("""list([[1. 1. 1. 0. 0. 0.],
 [0. 0. 0. 1. 1. 1.],
 [1. 1. 0. 0. 0. 1.],
 [0. 0. 1. 0. 1. 1.],
 [1. 1. 0. 1. 0. 0.],
 [0. 0. 1. 1. 1. 0.]])
""".replace(".", ","))
ensemble = matrix_to_ensemble(matrix)
print(hamming_loss(ytest, ensemble_predict(ensemble)))

0.010752688172043012
